In [ ]:
#import dependencies
import pandas as pd

In [ ]:
#open dataset

In [ ]:
#Check dataset consistency and stats

Primary analysis
================

District Summary
------------------------

In [ ]:
#Total schools


#Total students


#Total budget


#Average maths score


#Average reading score


#% passing maths


#% passing reading


#% overall passing


#Create summary table

School Summary
------------------------

In [ ]:
#Group by School Name


#Select columns


#Total students


#Total school budget


#Per student budget


#Average maths score


#Average reading score


#% Passing maths


#% passing reading


#% overall passing

Top performing schools
----------------------------------

In [ ]:
#Sort school summary


#Select top 5

Bottom performing schools
-------------------------------------

In [ ]:
#Sort school summary


#Select bottom 5


Secondary analysis
==================

Maths scores by grade
---------------------

Reading scores by grade
-----------------------

Scores by school spending
-------------------------

In [ ]:
#Define bins


#Select data


#Bin data

Scores by school size
---------------------

In [ ]:
#Define bins


#Select data


#Bin data

Scores by school type
---------------------

In [ ]:
#Define bins


#Select data


#Bin data

Reporting
=========